In [4]:
!pip install --upgrade scikit-learn


In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

# Step 1: Load Dataset
csv_path = "/content/wind_dataset.csv"
df = pd.read_csv(csv_path)

# Step 2: Data Preprocessing
df["DATE"] = pd.to_datetime(df["DATE"], errors='coerce')
df.interpolate(method='linear', inplace=True)  # Handle missing values

# Feature Engineering
df["TEMP_DIFF"] = df["T.MAX"] - df["T.MIN"]
features = ["RAIN", "T.MAX", "T.MIN", "T.MIN.G", "TEMP_DIFF"]
target = "WIND"

X = df[features].values
y = np.log1p(df[target].values)  # Apply log transformation to WIND speed

# Normalize the data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 3: Train the Model
model = MLPRegressor(
    hidden_layer_sizes=(50, 30),  # More layers for better feature learning
    activation='relu',
    solver='adam',
    max_iter=1000,  # Train longer for better accuracy
    learning_rate_init=0.005,  # Smaller learning rate for stability
    random_state=42
)
model.fit(X_train, y_train)

# Step 4: Evaluate Model
y_pred = np.expm1(model.predict(X_test))  # Convert predictions back from log scale
y_test_actual = np.expm1(y_test)  # Convert actual values back

mae = mean_absolute_error(y_test_actual, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred))
r2 = r2_score(y_test_actual, y_pred)

print(f" Model Performance:")
print(f" Mean Absolute Error (MAE): {mae:.3f}")
print(f" Root Mean Square Error (RMSE): {rmse:.3f}")
print(f" Model Accuracy (R² Score): {r2*100:.2f}%")

# Step 5: Save the Model and Scaler
os.makedirs("/content/wind_dataset", exist_ok=True)  # Create folder if not exists
model_filename = "/content/wind_dataset/mlp_model.pkl"
scaler_filename = "/content/wind_dataset/scaler.pkl"

joblib.dump(model, model_filename)
joblib.dump(scaler, scaler_filename)

# Verify if saved correctly
if os.path.exists(model_filename) and os.path.exists(scaler_filename):
    print(f" Model saved successfully at {model_filename}")
    print(f" Scaler saved successfully at {scaler_filename}")
else:
    print(f" Model saving failed! Check file paths.")

# Step 6: Allow Downloading Model
from google.colab import files
files.download(model_filename)
files.download(scaler_filename)


📊 Model Performance:
✅ Mean Absolute Error (MAE): 3.601
✅ Root Mean Square Error (RMSE): 4.639
✅ Model Accuracy (R² Score): 14.17%
✅ Model saved successfully at /content/wind_dataset/mlp_model.pkl
✅ Scaler saved successfully at /content/wind_dataset/scaler.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import joblib
import numpy as np

# Load the trained model and scaler
model_filename = "/content/wind_dataset/mlp_model.pkl"
scaler_filename = "/content/wind_dataset/scaler.pkl"

model = joblib.load(model_filename)
scaler = joblib.load(scaler_filename)

# User input
rain = float(input("Enter Rainfall (RAIN): "))
t_max = float(input("Enter Maximum Temperature (T.MAX): "))
t_min = float(input("Enter Minimum Temperature (T.MIN): "))
t_min_g = float(input("Enter Minimum Ground Temperature (T.MIN.G): "))  # Ask user for T.MIN.G

# Derived feature
temp_diff = t_max - t_min

# Prepare input data
input_data = np.array([[rain, t_max, t_min, t_min_g, temp_diff]])

# Normalize the input using StandardScaler
input_data_scaled = scaler.transform(input_data)

# Predict WIND speed and convert back from log scale
predicted_wind = np.expm1(model.predict(input_data_scaled))[0]

# Display the result
print(f"\n🌪️ Predicted Wind Speed: {predicted_wind:.2f}")


Enter Rainfall (RAIN): 5
Enter Maximum Temperature (T.MAX): 0.1
Enter Minimum Temperature (T.MIN): 21
Enter Minimum Ground Temperature (T.MIN.G): 11.1

🌪️ Predicted Wind Speed: 39.40
